# 05 · Evaluation Generation & Log-probs

Load Unsloth checkpoints, run evaluation prompts, and capture per-token statistics for MIA features.

In [ ]:
# Persistent Drive + run mode setup
import os
import sys
from pathlib import Path

try:
    from google.colab import drive  # type: ignore
    DRIVE_MOUNT = Path('/content/drive')
    if not DRIVE_MOUNT.exists():
        drive.mount('/content/drive')
except Exception as exc:  # pragma: no cover
    print(f'Colab drive mount skipped: {exc}')

if Path('/content/drive').exists():
    DRIVE_ROOT = Path('/content/drive/MyDrive').resolve()
else:
    DRIVE_ROOT = Path.home().resolve()

PROJECT_ROOT = DRIVE_ROOT / 'secure-llm-mia'
if not PROJECT_ROOT.exists():
    raise FileNotFoundError('Run 00_colab_setup.ipynb first to clone the repo on Drive.')

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

os.environ['SECURE_LLM_MIA_ROOT'] = str(PROJECT_ROOT)
os.chdir(PROJECT_ROOT)

from src.utils.runtime import current_run_mode

RUN_MODE = current_run_mode()
print('PROJECT_ROOT:', PROJECT_ROOT)
print('Active run mode:', RUN_MODE.name, '-', RUN_MODE.description)

DATA_ROOT = PROJECT_ROOT / 'data'
ARTIFACTS_DIR = PROJECT_ROOT / 'artifacts'
CHECKPOINT_ROOT = PROJECT_ROOT / 'checkpoints'
for path in (DATA_ROOT, ARTIFACTS_DIR, CHECKPOINT_ROOT):
    path.mkdir(parents=True, exist_ok=True)

BHC_DATA_DIR = DRIVE_ROOT / 'mimic-iv-bhc'
BHC_DATA_DIR.mkdir(parents=True, exist_ok=True)
BHC_CSV_PATH = BHC_DATA_DIR / 'mimic-iv-bhc.csv'
print('BHC CSV path:', BHC_CSV_PATH)


In [ ]:
import numpy as np
import torch
from unsloth import FastLanguageModel

from src.modeling.logprobs import token_level_stats

SLICE_ID = 1
TRACK = 'noreplay'
checkpoint_dir = CHECKPOINT_ROOT / f'slice_{SLICE_ID}' / TRACK / RUN_MODE.name
if not checkpoint_dir.exists():
    raise FileNotFoundError('Run notebook 04 to create checkpoints first.')

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=str(checkpoint_dir),
    max_seq_length=4096,
    load_in_4bit=True,
)
FastLanguageModel.for_inference(model)


In [ ]:
prompts = [
    'Summarize the hospital course for a patient treated for pneumonia.',
    'Describe follow-up instructions after ICU discharge.',
]
inputs = tokenizer(prompts, return_tensors='pt', padding=True)
with torch.inference_mode():
    outputs = model(**inputs)
logits = outputs.logits.detach().cpu().numpy()
target_ids = inputs['input_ids']
stats = token_level_stats(logits, target_ids)
print('Available features:', stats.keys())


📦 Persist per-example features to `reports/features_slice_t.parquet` for downstream attacks once evaluation prompts are finalized.